In [2]:
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import time
import glob
from lxml.html import parse
from sphfile import SPHFile
import pydub
import audiosegment
import pandas as pd
from collections import Counter
from bs4 import BeautifulSoup
import sys
import os
from tqdm.notebook import tqdm
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)
sr = 16000
dropout = 0.3
half = False

In [3]:
device = 3
torch.cuda.set_device(device)


In [4]:
class OverlayDataSet(torch.utils.data.Dataset):
    def __init__(self, csv, compute_original = False):
        super().__init__()
        self.overlays = pd.read_csv(csv)
        self.speakers = list(set(self.overlays['first_speaker']).union(set(self.overlays['second_speaker'])))
        self.speakers.sort()
        self.spkr2idx = {spkr:i for i, spkr in enumerate(self.speakers)}
        self.compute_original = compute_original
    def __len__(self):
        return len(self.overlays)
    def __getitem__(self, idx):
        overlay = self.overlays.iloc[idx]
        first_segment = np.load(overlay['first_file'])/(2**15)
        second_segment = np.load(overlay['second_file'])/(2**15)
        #padding to compensate rounding errors
        if len(first_segment)>len(second_segment):
            padding = np.zeros(len(first_segment)-len(second_segment))
            second_segment = np.concatenate((second_segment, padding))
        
        if len(first_segment)<len(second_segment):
            padding = np.zeros(len(second_segment)-len(first_segment))
            first_segment = np.concatenate((first_segment, padding))
        
        
        first_idx  = self.spkr2idx[overlay['first_speaker']]
        second_idx = self.spkr2idx[overlay['second_speaker']]
        target = np.zeros(len(self.speakers))
        target[first_idx] = 1.0
        target[second_idx] = 1.0
        if self.compute_original:
            return self.make_spectrogram(first_segment), self.make_spectrogram(second_segment),\
                self.make_spectrogram(first_segment+second_segment), target
        else:
            return self.make_spectrogram(first_segment+second_segment), target
    def make_spectrogram(self, segment):
        segment = segment[50:-50] # make size 200
        S = librosa.feature.mfcc(segment, sr=16000, n_mfcc=20, dct_type=2, n_fft = 1024, hop_length = 160)[1:14].T
        # 200*13
        S1 = np.diff(S)
        S2 = np.diff(S1)
        S = np.concatenate((S, S1, S2), axis = -1)
        return S
    
trainset = OverlayDataSet('overlay-train.csv', False)
valset = OverlayDataSet('overlay-val.csv', False)
testset = OverlayDataSet('overlay-test.csv', False)
print(trainset.speakers)
spec3, target = trainset[0]
plt.figure(figsize = (20, 6))
if trainset.compute_original:
    plt.subplot(131)
    plt.imshow(spec1[0].T)
    plt.subplot(132)
    plt.imshow(spec2[0].T)
    plt.subplot(133)
    plt.imshow(spec3[0].T)

['andrea_arsenault', 'brian_lamb', 'csp_waj_susan', 'david_brancaccio', 'eddie_mair', 'joie_chen', 'kathleen_kennedy', 'leon_harris', 'linda_wertheimer', 'linden_soles', 'lisa_mullins', 'lou_waters', 'lynn_vaughan', 'mark_mullen', 'natalie_allen', 'noah_adams', 'peter_jennings', 'robert_siegel', 'ted_koppel', 'thalia_assuras']


<Figure size 1440x432 with 0 Axes>

## Maybe try drastically increasing channel number in residual attention stage to see if it overfits

In [5]:
num_heads_2 = 4 # MHA heads


class Baseline(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn = nn.LayerNorm(36)
        self.reshape =  Lambda(lambda x: x.permute((1, 0, 2))) # L * batch_size * (num_heads*128)
        self.lstm = nn.LSTM(36, 32, 2, batch_first = False, bidirectional = True, dropout = dropout) # L * batch_size * 200 * n_hidden
        self.fc1 = nn.Linear(64, 32)
        self.average = Lambda(lambda x: x.mean(dim = 0)) # batch * n_hidden
        self.tanh = nn.Tanh()
        #self.norm = Lambda(lambda x: torch.nn.functional.normalize(x, p = 2, dim = 1)) # L2 normalize across n_hidden
        self.fc2 = nn.Linear(32, 20)
        self.sigmoid = nn.Sigmoid()
    def forward(self, X):
        X = self.bn(X)
        X = self.reshape(X)
        X, _ = self.lstm(X)
        X = self.fc1(X)
        X = self.average(X)
        X = self.tanh(X)
        X = self.fc2(X)
        X = self.sigmoid(X)
        return X
    
    
overnet = Baseline().cuda(device)
    
# tune hidden layers smaller if overfit
optimizer = torch.optim.Adam(overnet.parameters(), 0.001)

if os.path.exists('models/lstm.pth'):
    print('load model')
    checkpoint = torch.load('models/lstm.pth')
    overnet.load_state_dict(checkpoint['model_state_dict'])
    try:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    except:
        print('cannot load optimizer')
    loss = checkpoint['loss']
    if 'bestacc' in checkpoint:
        bestacc = checkpoint['bestacc']
    else:
        bestacc = 0.0
else:
    print('initializing new model')
    bestacc = 0.0
    
if half:
    overnet.half()  # convert to half precision
    for layer in overnet.modules():
        if isinstance(layer, nn.BatchNorm2d):
            layer.float()
            
overnet.train()
'bestacc:', bestacc

initializing new model


('bestacc:', 0.0)

## Also Do metrics on hitting a single person right

In [6]:
def find_max2(tensor):
    array = tensor.cpu().detach().numpy()
    max2 = []
    for row in array:
        max2.append(np.argsort(row)[::-1][:2])
    return np.array(max2)

def compute_corrects(tensor1, tensor2):
    max_1, max_2 = find_max2(tensor1), find_max2(tensor2)
    batch_size = max_1.shape[0]
    batch_corrects = 0
    for i in range(batch_size):
        if Counter(max_1[i])==Counter(max_2[i]):
            batch_corrects+=1
    return batch_corrects

batch_size = 32
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory = True, num_workers = 16)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True, pin_memory = True, num_workers = 16)
criterion = torch.nn.BCELoss()

for epoch in range(64):
    running_loss = 0.0
    running_accuracy = 0.0
    for batch_idx, (spec, target) in enumerate(tqdm(trainloader)):
        optimizer.zero_grad()
        spec, target = spec.float(), target.float()
        if half:
            spec, target = spec.half(),target.half()
        spec = spec.cuda(device)
        target = target.cuda(device)

        out = overnet(spec)
        loss = criterion(out, target)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(overnet.parameters(), 0.5)
        optimizer.step()
                
        running_loss += loss.item()
        running_accuracy += compute_corrects(out, target)/batch_size
        if batch_idx % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f accuracy: %.3f' % 
                  (epoch + 1, batch_idx + 1, running_loss / 200, running_accuracy / 200))
            running_loss = 0.0
            running_accuracy = 0.0
            torch.save({
            'model_state_dict': overnet.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, 'models/lstm.pth')

        
        
    corrects = 0
    for batch_idx, (spec, target) in enumerate(tqdm(valloader)):
        spec, target = spec.float(), target.float()
        if half:
            spec, target = spec.half(), target.half()
        spec = spec.cuda(device)
        target = target.cuda(device)
        overnet.eval()
        out = overnet(spec) 
        corrects += compute_corrects(out, target)
    print('val acc:', corrects/len(valset))
    if corrects/len(valset) > bestacc:
        bestacc = corrects/len(valset)
        torch.save({
        'model_state_dict': overnet.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'bestacc': bestacc
        }, 'models/best-lstm.pth')
    overnet.train()
    pass

[1,   200] loss: 0.369 accuracy: 0.007
[1,   400] loss: 0.325 accuracy: 0.005
[1,   600] loss: 0.324 accuracy: 0.007
[1,   800] loss: 0.314 accuracy: 0.010
[1,  1000] loss: 0.307 accuracy: 0.013
[1,  1200] loss: 0.302 accuracy: 0.021
[1,  1400] loss: 0.287 accuracy: 0.039
[1,  1600] loss: 0.265 accuracy: 0.071
[1,  1800] loss: 0.248 accuracy: 0.102
[1,  2000] loss: 0.240 accuracy: 0.121
[1,  2200] loss: 0.233 accuracy: 0.134
[1,  2400] loss: 0.224 accuracy: 0.159
[1,  2600] loss: 0.217 accuracy: 0.181




val acc: 0.2013611615245009


[2,   200] loss: 0.208 accuracy: 0.211
[2,   400] loss: 0.202 accuracy: 0.221
[2,   600] loss: 0.197 accuracy: 0.243
[2,   800] loss: 0.194 accuracy: 0.246
[2,  1000] loss: 0.190 accuracy: 0.271
[2,  1200] loss: 0.187 accuracy: 0.275
[2,  1400] loss: 0.185 accuracy: 0.292
[2,  1600] loss: 0.180 accuracy: 0.300
[2,  1800] loss: 0.177 accuracy: 0.327
[2,  2000] loss: 0.174 accuracy: 0.333
[2,  2200] loss: 0.173 accuracy: 0.329
[2,  2400] loss: 0.169 accuracy: 0.356
[2,  2600] loss: 0.167 accuracy: 0.368




val acc: 0.37250453720508164


[3,   200] loss: 0.161 accuracy: 0.376
[3,   400] loss: 0.160 accuracy: 0.382
[3,   600] loss: 0.159 accuracy: 0.390
[3,   800] loss: 0.156 accuracy: 0.399
[3,  1000] loss: 0.154 accuracy: 0.404
[3,  1200] loss: 0.151 accuracy: 0.425
[3,  1400] loss: 0.149 accuracy: 0.432
[3,  1600] loss: 0.149 accuracy: 0.430
[3,  1800] loss: 0.148 accuracy: 0.430
[3,  2000] loss: 0.145 accuracy: 0.453
[3,  2200] loss: 0.144 accuracy: 0.453
[3,  2400] loss: 0.142 accuracy: 0.469
[3,  2600] loss: 0.141 accuracy: 0.470



...

[34,   200] loss: 0.061 accuracy: 0.800
[34,   400] loss: 0.063 accuracy: 0.788
[34,   600] loss: 0.063 accuracy: 0.790
[34,   800] loss: 0.063 accuracy: 0.782
[34,  1000] loss: 0.063 accuracy: 0.784
[34,  1200] loss: 0.065 accuracy: 0.782
[34,  1400] loss: 0.062 accuracy: 0.797
[34,  1600] loss: 0.065 accuracy: 0.781
[34,  1800] loss: 0.065 accuracy: 0.781
[34,  2000] loss: 0.063 accuracy: 0.786
[34,  2200] loss: 0.065 accuracy: 0.780
[34,  2400] loss: 0.065 accuracy: 0.781
[34,  2600] loss: 0.065 accuracy: 0.775




val acc: 0.7393829401088929


[35,   200] loss: 0.061 accuracy: 0.794
[35,   400] loss: 0.065 accuracy: 0.786
[35,   600] loss: 0.061 accuracy: 0.796
[35,   800] loss: 0.064 accuracy: 0.785
[35,  1000] loss: 0.062 accuracy: 0.794
[35,  1200] loss: 0.064 accuracy: 0.792
[35,  1400] loss: 0.063 accuracy: 0.785
[35,  1600] loss: 0.062 accuracy: 0.792
[35,  1800] loss: 0.064 accuracy: 0.785
[35,  2000] loss: 0.063 accuracy: 0.797
[35,  2200] loss: 0.064 accuracy: 0.785
[35,  2400] loss: 0.063 accuracy: 0.789
[35,  2600] loss: 0.066 accuracy: 0.776




val acc: 0.7406533575317604


KeyboardInterrupt: 

In [8]:
checkpoint = torch.load('models/best-lstm.pth')
overnet.load_state_dict(checkpoint['model_state_dict'])

testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True, pin_memory = True, num_workers = 16)
        
corrects = 0
for batch_idx, (spec, target) in enumerate(tqdm(testloader)):
    spec, target = spec.float(), target.float()
    if half:
        spec, target = spec.half(), target.half()
    spec = spec.cuda(device)
    target = target.cuda(device)
    overnet.eval()
    out = overnet(spec) 
    corrects += compute_corrects(out, target)
print('test acc:', corrects/len(testset))
overnet.train()
pass


test acc: 0.7449183303085299
